In [1]:
import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time
import pandas as pd

In [2]:
# Get authentication credentials to initialize ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AbUR2VMQsIzFCQvr1qHPl-S455nt-xH561xk37QblGwLECMopZhp6XgIFXQ

Successfully saved authorization token.


In [29]:
# Define geometry for data collection


# # Nanda Bet larger S1 extent
# roi = ee.Geometry.Rectangle([69.98568, 22.66953, 72.76651, 24.60271]);
# shp_file = 'data/s1_boundary.shp'
# boundary = geemap.shp_to_ee(shp_file)


# # Nanda Bet smaller region
# roi = ee.Geometry.Rectangle([70.7980547394194417, 23.2880385161501735, 71.5060515087354958, 23.9024834537972986])  
# shp_file = 'data/smaller_region.shp'
# boundary = geemap.shp_to_ee(shp_file)


# Mississippi River at KY, TN, MO
roi = ee.Geometry.Rectangle([-89.8, 36.2, -89.2, 36.8])

# lon_limits = [-89.783     -89.2071];
# lat_limits = [36.2568      36.7291];


In [43]:
# Get ERA5 hourly precipitation in ROI

d1 = '2018-08-01'
d2 = '2018-08-02'
#dataset = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filter(ee.Filter.date(d1, d2)).filterBounds(boundary.geometry())
dataset = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filter(ee.Filter.date(d1, d2)).filterBounds(roi)
                
# compute sum to get daily precipitation
#daily_precip = dataset.select('total_precipitation').sum().clip(boundary.geometry())
daily_precip = dataset.select('total_precipitation').sum().clip(roi)


In [44]:
print(daily_precip)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.clip",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    -89.8,
                    36.8
                  ],
                  [
                    -89.8,
                    36.2
                  ],
                  [
                    -89.2,
                    36.2
                  ],
                  [
                    -89.2,
                    36.8
                  ]
                ]
              ]
            },
            "evenOdd": {
              "constantValue": true
            }
          }
        }
      },
      "input": {
        "functionInvocationValue": {
          "functionName": "reduce.sum",
          "arguments": {
            "collection": {
            

In [14]:
# Define visualization parameters

precip_vis = {
  'bands': ['total_precipitation'],
  'min': 0,
  'max': 0.2,
  'palette': [
    "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
    "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
    "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",
  ],
  'opacity': 0.5
};


In [45]:
# Create map and add layers

M = geemap.Map()
M.centerObject(roi, 10)
M.addLayer(daily_precip,precip_vis,'Precipitation')
M

Map(center=[36.499988078482595, -89.50000000000023], controls=(WidgetControl(options=['position', 'transparent…

In [55]:
# Export daily precipitation total 

d = pd.date_range(start='2018-08-01',end='2018-08-09')
ns = daily_precip.select('total_precipitation').projection().nominalScale().getInfo()
task_id = []

for i in range(len(d)-1):
    d1 = d[i].strftime('%Y-%m-%d')
    d2 = d[i+1].strftime('%Y-%m-%d')
    
#     dataset = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filter(ee.Filter.date(d1, d2)).filterBounds(boundary.geometry())
#     daily_precip = dataset.select('total_precipitation').sum().clip(boundary.geometry())
    
    dataset = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filter(ee.Filter.date(d1, d2)).filterBounds(roi)
    daily_precip = dataset.select('total_precipitation').sum().clip(roi)
    
    txt = 'era5_daily_precip_sum_meters_roi_' + d1
    task = ee.batch.Export.image.toDrive(image=daily_precip,
                                         description=txt,
                                         scale=ns,
#                                          region=boundary.geometry(),
                                         region=roi,
                                         fileNamePrefix=txt,
                                         crs='EPSG:4326',
                                         fileFormat='GeoTIFF',
                                         maxPixels=1e9)
    task.start()
    task_id.append(task.id)
    print(d1)



    

2018-08-01
2018-08-02
2018-08-03
2018-08-04
2018-08-05
2018-08-06
2018-08-07
2018-08-08


In [57]:
task.list()

[<Task LD4C6EIK7ORP3OSBU4R74UFT EXPORT_IMAGE: era5_daily_precip_sum_meters_roi_2018-08-08 (COMPLETED)>,
 <Task 6QIOSE43WTB6ABE2BIGWF7GJ EXPORT_IMAGE: era5_daily_precip_sum_meters_roi_2018-08-07 (RUNNING)>,
 <Task GX7GN35P7P52PKYNMNHY7CYD EXPORT_IMAGE: era5_daily_precip_sum_meters_roi_2018-08-06 (COMPLETED)>,
 <Task SJA44GF3JEXAK5CLB5MZA3OC EXPORT_IMAGE: era5_daily_precip_sum_meters_roi_2018-08-05 (COMPLETED)>,
 <Task 5FUOIW4DOK4TJJ4MVIJOF6K3 EXPORT_IMAGE: era5_daily_precip_sum_meters_roi_2018-08-04 (COMPLETED)>,
 <Task JNVLFCZPVV2F54LMPWH3BOHL EXPORT_IMAGE: era5_daily_precip_sum_meters_roi_2018-08-03 (COMPLETED)>,
 <Task PWDYOFUOFG2CNBQRGTBS3UYS EXPORT_IMAGE: era5_daily_precip_sum_meters_roi_2018-08-02 (RUNNING)>,
 <Task 4SQLQUINLNXAZML5T7KV4M3C EXPORT_IMAGE: era5_daily_precip_sum_meters_roi_2018-08-01 (COMPLETED)>,
 <Task J74NV3M55X5MXBV4BXPYEDX3 EXPORT_IMAGE: era5_daily_precip_sum_meters_roi_2021-12-31 (COMPLETED)>,
 <Task Y2ETPP4TQPUCZELCZ37GV576 EXPORT_IMAGE: era5_daily_precip_sum_